# SSPs Calibration: Benveniste

Our working local folders for this calibration are contained in the parent folder of this notebook, in _calibration/Benveniste_

The authors of Benveniste et al., 2020 provided us with three data files in the _data_ subfolder, including _all_ssp_em.csv_, _all_ssp_gdp.csv_, and _all_ssp_pop.csv_, which we will post process below.

In [1]:
using CSVFiles, Query, DataFrames, Missings

## Socioeconomic Data

In [2]:
gdp_df = load(joinpath(@__DIR__, "data", "all_ssp_gdp.csv")) |> DataFrame
pop_df = load(joinpath(@__DIR__, "data", "all_ssp_pop.csv")) |> DataFrame

for scen in unique(gdp_df.scen)
    pop_data = pop_df |> @filter(_.scen == scen) |> DataFrame
    gdp_data = gdp_df |> @filter(_.scen == scen) |> DataFrame
    
    # check that keys are the same
    pop_data.period == gdp_data.period
    pop_data.country == gdp_data.country

    df = DataFrame(:year => pop_data.period, 
        :country => pop_data.country, 
        :pop => pop_data.pop ./ 1e3, # convert thousand to million
        :gdp => gdp_data.gdp # billion of USD in $2005
    ) |> save(joinpath(@__DIR__, "..", "..", "data", "socioeconomic", "Benveniste_$(scen).csv"))
end

## Emissions Data

Note that for emissions, Benveniste et al., 2020 provides emissions at the country level but here we aggregate to global to fit the expected format of our components.  In addition, Benveniste et al., 2020 only provides CO2 emissions, not CH4, SF6, or N2O, so this will be handled by the components.

In [4]:
em_df = load(joinpath(@__DIR__, "data", "all_ssp_em.csv")) |> DataFrame

scenario_dict = Dict(
    "SSP1-1.9-SPA1" => "SSP119",
    "SSP2-4.5-SPA2" => "SSP245",
    "SSP3-7.0-SPA0" => "SSP370",
    "SSP4-6.0-SPA4" => "SSP460",
    "SSP5-8.5"      => "SSP585"
)

for scenario in unique(em_df.scenario)
    println("Working on emissions for $scenario ...")
    df = DataFrame(:year => [], :carbon_dioxide => [])
    for year in unique(em_df.period)
        em_data = em_df |> 
            @filter(_.scenario == scenario && _.period == year) |> 
            @select(:em) |> DataFrame

        # sum over countries and convert from Mt CO2 to Gt C
        total_em = 12/44 .* sum(skipmissing(em_data.em)) ./ 1e3 

        append!(df, DataFrame(:year => year, :carbon_dioxide => total_em)) # sum over countries and convert 
    end
    df |> save(joinpath(@__DIR__, "..", "..", "data", "emissions", "Benveniste_$(scenario_dict[scenario]).csv"))
end

Working on emissions for SSP1-1.9-SPA1 ...
Working on emissions for SSP2-4.5-SPA2 ...
Working on emissions for SSP3-7.0-SPA0 ...
Working on emissions for SSP4-6.0-SPA4 ...
Working on emissions for SSP5-8.5 ...
